# Loading Libraries

In [1]:
# Pandas, Numpy
import pandas as pd
import numpy as np 

# Tensorflow
import tensorflow as tf 

# RBM from Recommenders
from recommenders.models.rbm.rbm import RBM 
from recommenders.utils.timer import Timer 
from recommenders.utils.plot import line_graph 

# Affinity Matrix 
from recommenders.datasets.sparse import AffinityMatrix

# Evaluation 
from recommenders.evaluation.python_evaluation import (
    precision_at_k,
    recall_at_k
)
from sklearn.metrics import accuracy_score 

# Visualization
from matplotlib import pyplot as plt 

# Import & Read Dataset

In [2]:
# Read Train & Test Data
train = pd.read_csv("../../00_Data/rec_sys_ratings_train.csv", index_col=0)
test = pd.read_csv("../../00_Data/rec_sys_ratings_test.csv", index_col=0)

In [3]:
# Check Shapes
print("Shape of Train:", train.shape)
print("Shape of Test: \t", test.shape)

Shape of Train: (393334, 3)
Shape of Test: 	 (98401, 3)


In [13]:
# Check Number of Unique Items and User in Train & Test 
print("Unique Users in Train:", train.CustomerID.nunique())
print("Unique Users in Test:", test.CustomerID.nunique())
print("Unique Items in Train:", train.StockCode.nunique())
print("Unique Items in Test:", test.StockCode.nunique())

Unique Users in Train: 3011
Unique Users in Test: 2978
Unique Items in Train: 2330
Unique Items in Test: 2330


In [5]:
# Value Counts of Ratings in Train
train.purchased.value_counts()

-1    154381
 1    106576
-2    101061
 2     31316
Name: purchased, dtype: int64

In [6]:
# Value Counts of Ratings in Test
test.purchased.value_counts()

-1    38596
 1    26702
-2    25265
 2     7838
Name: purchased, dtype: int64

# Exclude negative Ratings for Modeling

In [7]:
# Exclude negative Ratings from Train  
train = train[train['purchased']>0]

# Show Impact 
train.purchased.value_counts()

1    106576
2     31316
Name: purchased, dtype: int64

In [8]:
# Exclude negative Ratings from test  
test = test[test['purchased']>0]

# Show Impact 
test.purchased.value_counts()

1    26702
2     7838
Name: purchased, dtype: int64

In [15]:
# Ensure that the same items and users are in both sets 
# Find the set of unique items in both the train and test sets
train_users = set(train['CustomerID'].unique())
test_users = set(test['CustomerID'].unique())

# Find the intersection of the sets from step 1
common_items = train_users.intersection(test_users)

# Filter the train and test sets to include only the rows with item IDs that are in the intersection set
train = train[train['CustomerID'].isin(common_items)]
test = test[test['CustomerID'].isin(common_items)]

In [16]:
# Check Number of Unique Items and User in Train & Test 
print("Unique Users in Train:", train.CustomerID.nunique())
print("Unique Users in Test:", test.CustomerID.nunique())
print("Unique Items in Train:", train.StockCode.nunique())
print("Unique Items in Test:", test.StockCode.nunique())

Unique Users in Train: 2978
Unique Users in Test: 2978
Unique Items in Train: 2330
Unique Items in Test: 2330


# Preparation of Datasets for Modeling 

In [17]:
# Set standard names for the analysis 
header = {
        "col_user": "CustomerID",
        "col_item": "StockCode",
        "col_rating": "purchased",
    }

# Instantiate the sparse matrix generation  
am_train = AffinityMatrix(df = train, col_user='CustomerID', col_item='StockCode', col_rating='purchased')
am_test = AffinityMatrix(df = test, col_user='CustomerID', col_item='StockCode', col_rating='purchased')

# Obtain the sparse matrix 
Xtr, _, _ = am_train.gen_affinity_matrix()
Xtst, _, _ = am_test.gen_affinity_matrix()

In [18]:
# Sanity Check: Print Shape of Train & Test Matrix - Do numbers match nr of unique users and items from above 
print(Xtr.shape)
print(Xtst.shape) 

(2978, 2330)
(2978, 2330)


In [19]:
# Sanity check: Xtr & Xtst After reloading & Transformation
print(np.unique(Xtr))
print(np.unique(Xtst))

[0 1 2]
[0 1 2]


# Modeling

## Hyperparameter Tuning 1 

In [20]:
# Set up Parameters 
hidden_neurons = [500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500]
batch = [200, 300]
lrate = [0.002, 0.003, 0.004]

# Initiate Lists for Assessment
recall_10 = []
recall_20 = []
precision_10 = []
precision_20 = []
neurons = []
batches = []
learning_rates = []

# Loop through lists of parameters 

for n in hidden_neurons:
    for b in batch:
        for l in lrate:
            # Delete Model First
            try:
                del(model)
            except:
                pass
            
            # Define Model with Parameters
            model = RBM(
                possible_ratings=np.setdiff1d(np.unique(Xtr), np.array([0])), # Always provide this range - way better results! 
                visible_units=Xtr.shape[1],
                hidden_units=n,
                training_epoch=30,
                minibatch_size=b,
                keep_prob=0.7,
                with_metrics=True,
                learning_rate=l,
                seed=42
            )
            
            # Fit Model 
            model.fit(Xtr)
            
            # Recommend top k
            top_k = model.recommend_k_items(Xtst, top_k=20, remove_seen=True)
            
            # Map Back Xtst & Top K
            top_k_df = am_test.map_back_sparse(top_k, kind = 'prediction')
            test_df  = am_test.map_back_sparse(Xtst, kind='ratings')
            
            # Evaluation 
            recall_at_ten  = recall_at_k(test_df, top_k_df, col_user="CustomerID", col_item="StockCode", 
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=10)
            recall_at_twenty = recall_at_k(test_df, top_k_df, col_user="CustomerID", col_item="StockCode", 
                                    col_rating="purchased", col_prediction="prediction", 
                                    relevancy_method="top_k", k=20)
            precision_at_ten = precision_at_k(test_df, top_k_df, col_user='CustomerID', col_item='StockCode',
                                            col_rating ='purchased', col_prediction="prediction",
                                            relevancy_method='top_k', k = 10)
            precision_at_twenty = precision_at_k(test_df, top_k_df, col_user='CustomerID', col_item='StockCode',
                                col_rating ='purchased', col_prediction="prediction",
                                relevancy_method='top_k', k = 20)
            
            # Append lists 
            recall_10.append(recall_at_ten)
            recall_20.append(recall_at_twenty)
            precision_10.append(precision_at_ten)
            precision_20.append(precision_at_twenty)
            neurons.append(n)
            batches.append(b)
            learning_rates.append(l)
            
            # Print Results 
            print("Recall at 10 for ", n, "neurons:", recall_at_ten)

Recall at 10 for  500 neurons: 0.03655464232159039
Recall at 10 for  500 neurons: 0.039258959894461404
Recall at 10 for  500 neurons: 0.039531813390350695
Recall at 10 for  500 neurons: 0.038976471513312655
Recall at 10 for  500 neurons: 0.03829695683567469
Recall at 10 for  500 neurons: 0.03838612289026063
Recall at 10 for  600 neurons: 0.0492460132163925
Recall at 10 for  600 neurons: 0.04785465058832121
Recall at 10 for  600 neurons: 0.055045790677051766
Recall at 10 for  600 neurons: 0.0480472102992972
Recall at 10 for  600 neurons: 0.05090085316031563
Recall at 10 for  600 neurons: 0.05193373079419404
Recall at 10 for  700 neurons: 0.06945121473087952
Recall at 10 for  700 neurons: 0.06480616233689063
Recall at 10 for  700 neurons: 0.06803969410846711
Recall at 10 for  700 neurons: 0.07302058479314877
Recall at 10 for  700 neurons: 0.07087918228558795
Recall at 10 for  700 neurons: 0.07257915603382785
Recall at 10 for  800 neurons: 0.06215261917828008
Recall at 10 for  800 neurons

In [21]:
# Create Dataframe out of the Results of the Hyperparameter Tuning 
df_tuning_1 = pd.DataFrame({
    "hidden_neurons": neurons,
    "minibatches": batches,
    "learning_rates":learning_rates,
    "recall@10": recall_10,
    "recall@20": recall_20,
    "precision@10":precision_10,
    "precision@20":precision_20
})
# Sort Values by recall@10 
df_tuning_1.sort_values(by='recall@10', ascending=False)

,hidden_neurons,minibatches,learning_rates,recall@10,recall@20,precision@10,precision@20
38,1100,200,0.004,0.187308,0.255165,0.192109,0.133479
50,1300,200,0.004,0.172797,0.244212,0.183983,0.130725
37,1100,200,0.003,0.170832,0.240730,0.177099,0.126343
49,1300,200,0.003,0.165222,0.233686,0.175655,0.125856
41,1100,300,0.004,0.151203,0.211843,0.155473,0.111182
...,...,...,...,...,...,...,...
1,500,200,0.003,0.039259,0.065691,0.043956,0.036249
3,500,300,0.002,0.038976,0.064003,0.042478,0.034637
5,500,300,0.004,0.038386,0.063176,0.043956,0.035645
4,500,300,0.003,0.038297,0.062515,0.043855,0.035124
